# Lesson 11: The Full Pipeline

In the last 2 lessons, we built each agent separately. Now it's time to **connect everything** into a complete pipeline:

```
Topic --> Research --> Outline --> Writer --> Image (optional) --> Finished article
```

This pipeline is the **actual product code**. We'll import directly from the source, not rewrite it.

What's different from running agents individually:
- **Database tracking** — each step updates the article's status in SQLite
- **Error handling** — if any step fails, status changes to `error` with a detailed message
- **File output** — the finished article is automatically saved as a `.md` file

## Pipeline Diagram

Here's the full flow with database status tracking:

```
create_article(topic)
      |
      v
  [queued]  -->  [researching/outlining]  -->  [writing]  -->  [enriching]  -->  [review]
                  Research Agent              Writer Agent     Image Agent        Done!
                  + Outline Agent             (Grok-4)         (optional)
                  (Claude Sonnet)                              (Claude Sonnet)
                        |                         |                  |
                        v                         v                  v
                  On error: [error]         On error: [error]  On error: [error]
```

At each step:
1. Update status in the database
2. Run the agent
3. Save results and move to the next step

If any step fails, the pipeline stops and saves the `error_message` to the database.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath("../../output"))

from dotenv import load_dotenv
load_dotenv()

# Import the real pipeline from our product!
from pipeline import run_content_pipeline
from db import create_article, get_article, init_db

init_db()

## What Does `run_content_pipeline()` Do?

This function is the **heart** of the product. It runs 4 steps:

1. **Research** — Calls Research Agent, saves research notes
2. **Outline** — Calls Outline Agent with the research notes, saves ContentOutline (JSON)
3. **Write** — Calls Writer Agent with the outline, saves the Markdown article
4. **Enrich** (optional) — If Image Agent exists, finds and inserts images

Between each step, it updates the database:
```python
update_article_status(article_id, "writing", outline=outline_json)
```

On error:
```python
update_article_status(article_id, "error", error_message=str(e))
```

Finally, the article is saved to a file and the status changes to `review`.

## Error Handling: try / except

What happens when something goes wrong? An API call fails, a search returns nothing, or the network drops.

Python uses **try/except** to handle errors gracefully:

```python
try:
    # Try running this code
    result = agent.run("...")
except Exception as e:
    # If it fails, do this instead
    print(f"Something went wrong: {e}")
```

- Code inside `try` runs normally
- If any line inside `try` fails, Python **jumps** to `except` immediately
- The variable `e` contains the error message
- Without try/except, the whole program would crash

Our pipeline wraps all 4 agent steps in try/except. If any step fails:
1. The error message is saved to the database (`error_message` column)
2. The article status changes to `"error"`
3. The pipeline stops and returns `False`

This means **one failing article doesn't crash the whole batch** — other articles continue processing normally.

> **Cost:** The next cell runs the full pipeline (~$1-3 per run: Sonnet for research + outline, Grok for writing). Takes 2-4 minutes. Run it once.

In [ ]:
topic = "10 Simple SEO Tips for Beginners"

# Create article record in database
article_id = create_article(topic)
print(f"Created article #{article_id}\n")

# Run the full pipeline
success = run_content_pipeline(article_id, topic)
print(f"\nSuccess: {success}")

In [ ]:
article = get_article(article_id)
print(f"Article #{article['id']}")
print(f"  Topic:  {article['topic']}")
print(f"  Status: {article['status']}")
print(f"  Words:  {article['word_count']}")
print(f"  File:   {article['output_file']}")

In [ ]:
if article.get("output_file"):
    with open(article["output_file"], "r", encoding="utf-8") as f:
        content = f.read()
    print(content[:2000] + "\n\n... (truncated for display)")

## Exercise

Using the `article` dict from above, write code to check the article quality:

1. Is the word count above 1000? Print a pass/fail message.
2. What status is the article in? (should be `"review"` if successful)
3. Does the output file exist on disk? (hint: `os.path.exists(article['output_file'])`)

Bonus: Use `list_articles` from `db` to see all articles in the database and count how many are in each status.

In [ ]:
# Exercise: Write your code here


## Congratulations!

You've built a complete **AI-powered SEO content pipeline**:

- **4 specialized agents**, each using the best model and tools for its task
- **Database tracking** — monitor status from `queued` through `review`
- **Error handling** — pipeline stops safely on errors
- **File output** — articles automatically saved as Markdown files

This code runs **exactly the same** in the real product. When you run:
```bash
python output/cli.py create "topic"
```
It calls the exact same `run_content_pipeline()` we just tested.

### Next: Module 4

In the next module, we'll turn this pipeline into a **complete product** with:
- Database layer for persistent storage
- CLI interface (command line)
- Chat interface (conversational AI team)
- Batch processing (create many articles at once)